In [2]:
import os
import requests
from bs4 import BeautifulSoup
import glob
import heapq
import bisect
import re
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# Daftar URL untuk di-scrape
urls = [
    "https://www.kompasiana.com/dian96997/64cbdf684addee3feb2fbbc2/komunikasi-tanpa-kata-posisi-budaya-visual-dalam-ilmu-komunikasi",
    "https://www.kompasiana.com/rizqiakbari4638/64a19d794addee41f4148982/iklan-dan-periklanan-di-era-digital",
    "https://www.kompasiana.com/pumacommunicationpresuniv0608/648d5f7208a8b55c7d7bc132/mahasiswa-ilmu-komunikasi-president-university-lakukan-kunjungan-ke-starvision",
    "https://www.kompasiana.com/kartiah367/6481fd4829f19e62881fc272/merangkul-keunikan-diri-memperkuat-self-esteem-melalui-komunikasi-antar-pribadi-yang-positif",
    "https://www.kompasiana.com/ekameiliyani/642fe6a44addee2b8f21b652/menjadi-pengusaha-muda-adalah-investasi-yang-tepat-untuk-masa-depan",
    "https://www.kompasiana.com/zainul16921/6504d2934addee305e68e6a2/transformasi-bisnis-langkah-menuju-sukses",
    "https://www.kompasiana.com/fitriawardani8538/65041b75fdf1c501c67dd972/upgrade-jaringan-komunikasi-modernisasi-media-komunikasi",
    "https://www.kompasiana.com/fitriawardani8538/650416cd08a8b515af672692/modernisasi-teknologi-mengubah-cara-berkomunikasi",
    "https://www.kompasiana.com/fitriawardani8538/6503941be3b78f636b59c9a2/perubahan-interaksi-sosial-online-modernisasi-dalam-komunikasi",
    "https://www.kompasiana.com/fitriawardani8538/65038f31e3b78f5fb61b3942/keamanan-digital-di-era-teknologi-modernisasi-teknologi-komunikasi",
    "https://www.kompasiana.com/muhamadqli/6501ca3a4addee175b290432/esperanto-bahasa-internasional-untuk-komunikasi-global",
    "https://www.kompasiana.com/fitriawardani8538/6502414f4addee11df477ce2/modernisasi-teknologi-chat-dan-pesan-singkat",
    "https://www.kompasiana.com/fitriawardani8538/6501c6e808a8b535a0453712/modernisasi-teknologi-perkembangan-media-sosial",
    "https://www.kompasiana.com/fitriawardani8538/6500f59f08a8b569c556b6c2/jaringan-kecepatan-tinggi-modernisasi-media-komunikasi",
    "https://www.kompasiana.com/fitriawardani8538/6500f331e1a1674c3c2a97e2/modernisasi-teknologi-tantangan-komunikasi-antarbudaya",
    "https://www.kompasiana.com/fitriawardani8538/6500f0b54addee57a914fb73/perubahan-pola-komunikasi-modernisasi-dalam-komunikasi",
    "https://www.kompasiana.com/syahrialsyahrial0585/64fd8f7e4addee55c96e9f82/melacak-jejak-mahasiswa-do-ketidakaktifan-komunikasi-dan-solusi",
    "https://www.kompasiana.com/fitriawardani8538/64cae0b108a8b503b77fd992/komunikasi-dalam-proses-negosiasi-mencapai-win-win-solution",
    "https://www.kompasiana.com/fitriawardani8538/64c50edd4addee302b6d3832/komunikasi-dan-teknik-negosiasi-yang-sukses-dalam-berbisnis",
    "https://www.kompasiana.com/fitriawardani8538/64c433bb08a8b57901143842/peran-komunikasi-dalam-pengembangan-dan-pemeliharaan-hubungan-pelanggan",
    "https://www.kompasiana.com/muhammadazkia11/64ad772a4addee3a2a41deb3/prinsip-prinsip-komunikasi-islam-dalam-dunia-bisnis",
    "https://www.kompasiana.com/dewilailypurnamasari/642acdb74addee6fec20d7c4/upgrade-skill-mahasiswa-kelas-komunikasi-dengan-jadi-kompasianer",
    "https://www.kompasiana.com/srirahmawati1809/63a6fccb906beb0c191ff8d2/keterampilan-komunilasi-menjadi-hal-utama-dalam-bisnis",
    "https://www.kompasiana.com/nurulhanifah/639ac3044addee2e4e04ba52/teknik-teknik-lobying-dalam-dunia-bisnis",
    "https://www.kompasiana.com/dewiani63893/63976efa29f19e1dde6c3ba2/pentingnya-komunikasi-bisnis-untuk-keberhasilan-suatu-usaha",
    "https://www.kompasiana.com/citrakumala9598/65068baa08a8b52785406332/mengurangi-jumlah-pengangguran-dan-umkm-dapat-naik-kelas",
    "https://www.kompasiana.com/yundriakhyar/6506a74b08a8b5041c1bf852/kasian-wanita-jika-tidak-berpendidikan-tinggi",
    "https://www.kompasiana.com/fabiohernanes8827/6506a72008a8b56cbc2e79c2/5-alasan-mengapa-bule-suka-tinggal-di-bali-visa-piknik-dan-pesona-pulau-dewata",
    "https://www.kompasiana.com/rukiyahbrrambe7773/6505b18f4addee474815ad12/khadijah-binti-khuwailit",
    "https://www.kompasiana.com/irsalefendi2414/6506a5e64addee5df8200fb2/belajar-demokrasi-sejak-bangku-sekolah-melalui-pemilihan-osis",
    "https://www.kompasiana.com/vincentiadevika/6506a98908a8b552d1169e53/the-ride-2-bukti-film-indonesia-mampu-bersaing-di-luar-negeri",
    "https://www.kompasiana.com/erfransdo/6503593f4addee3f792e9822/ketuk-tular-prinsip-masyarakat-kampung-adat-cireundeu-dalam-mencegah-pandemi-covid-19",
    "https://www.kompasiana.com/ngabilasalama/64f92eb1e2c0f97d913476c2/ditemukan-covid-19-pirola-ba-2-86-subvarian-omicron-12-negara-dunia-yuk-lengkapi-vaksinasi-segera",
    "https://www.kompasiana.com/putriyashar1043/64f6db9a12d50f1d33722813/peraturan-bank-indonesia-no-8-tahun-2023-langkah-penting-dalam-mengatasi-dampak-pandemi-covid-19",
    "https://www.kompasiana.com/jonathanvelim5489/64e3726e08a8b51a5a0406f2/pemanfaatan-pajak-rokok-dan-bea-cukai-untuk-meningkatkan-anggaran-kesehatan",
    "https://www.kompasiana.com/dheafitra/64a565a708a8b51911672ab2/covid-19-perkembangan-terkini-dan-tantangan-global-yang-masih-dihadapi",
    "https://www.kompasiana.com/ngabilasalama/6498ff2910d8e01a155aab63/endemi-covid-19-situasi-dki-jakarta-juni-2023",
    "https://www.kompasiana.com/edgarjeremy/6503532e4addee163f578842/kenapa-konten-judi-online-ada-di-website-pemerintahan",
    "https://www.kompasiana.com/kresnacybercell2725/6503ae2008a8b55abf62bf84/awasi-program-kerja-desa-inspektorat-gandeng-paguyuban-bpd",
    "https://www.kompasiana.com/darmawijaya9218/65011ccf08a8b534592ede73/kelompok-kkm-71-bantu-pelayanan-di-kantor-kelurahan-mekarsari",
    "https://www.kompasiana.com/ardibagusprasetyo/64f93a7e4addee1a7b1f0dc3/mengenal-menteri-termiskin-indonesia-bernama-ir-sutami",
    "https://www.kompasiana.com/suko56891/64f86c3f08a8b549357a1b42/dampak-pemerintahan-di-daerah-yang-ditinggalkan-kepala-daerah-yang-habis-masa-jabatannya",
    "https://www.kompasiana.com/kresnacybercell2725/64f467ea08a8b56b7e789852/gelar-pertemuan-rutin-paguyuban-anggota-bpd-se-kecamatan-demak-kota-jalin-sinergitas",
    "https://www.kompasiana.com/muhammadridho5816/6506894908a8b52dbf482ff2/dilema-teknologi-dalam-perawatan-kesehatan-rasionalitas-teknis-vs-etika-perawatan",
    "https://www.kompasiana.com/syahiduzzaman5860/65064b6d6e14f164906ec952/plagiarisme-tak-terlihat",
    "https://www.kompasiana.com/www.muhammadnurul010218.blogspot.com/6505aeb26e14f164d71f2cd2/gentrifikasi-masyarakat-kotakah-yang-berubah-atau-masyarakat-desa",
    "https://www.kompasiana.com/aghatax9912311/6505d3a94addee71fb0e6783/teknologi-etika-dan-implikasinya-di-indonesia-analisis-kritis-artikel-tech-ethics-speaking-ethics-to-power-or-power-speaking-ethics",
    "https://www.kompasiana.com/fuaidil/6505b7a808a8b55352437f22/menghadapi-era-cyber-otonom-implikasi-hukum-dan-etika-untuk-keamanan-nasional",
    "https://www.kompasiana.com/hafidz52970/650593a508a8b531047b6262/etika-penggunaan-teknologi-pemrosesan-wajah-dalam-sistem-keamanan-dan-identifikasi",
    "https://www.kompasiana.com/fitriawardani8538/65055ff408a8b522507d1e22/modernisasi-teknologi-privasi-dan-keamanan-digital",
    "https://www.kompasiana.com/kayla92662/64fe2362e1a16706cc3e4712/transformasi-mahasiswa-pasca-pandemi-menuju-era-society-5-0-berkembang-atau-tumbang",
    "https://www.kompasiana.com/nasywamutiara/650545b06e14f109585fca52/transformasi-mahasiswa-pasca-pandemi-menuju-era-society-5-0-berkembang-atau-tumbang",
    "https://www.kompasiana.com/radinamutiahaira9986/650519b108a8b5734f340153/chatgpt-lebih-dari-sekedar-bot-obrolan-menyelami-dunia-etika-teknologi-canggih",
    "https://www.kompasiana.com/roihanfs/6504aff34addee6fac0589b2/mengidentifikasi-ancaman-terhadap-infrastruktur-kritis-di-era-digital",
    "https://www.kompasiana.com/syahiduzzaman5860/6504ec746e14f1235c2e9622/mobil-listrik-penyelamat-lingkungan-atau-ancaman-tersembunyi",
    "https://www.kompasiana.com/universitasahmaddahlan/650695a66e14f17c015753c2/uad-terus-tingkatkan-kuota-penerimaan-mahasiswa-pada-program-pmm",
    "https://www.kompasiana.com/merzagamal8924/650691a36e14f112aa4e3c82/mengatasi-beban-administrasi-guru-untuk-meningkatkan-proses-pembelajaran",
    "https://www.kompasiana.com/nurtaufiikgfr4620/650684f96e14f11bde7b1992/belajar-mengenal-bahasa-isyarat-dari-taman-literasi-jakarta",
    "https://www.kompasiana.com/achmadseptiadi/650678cc6e14f11af341e882/dinamika-pembelajaran-agama-islam-di-era-digital-4-0",
    "https://www.kompasiana.com/silvyanalady0452/6504052dfdf1c5071e7aebe5/ppsdm-migas-gelar-pelatihan-operasi-produksi-untuk-pt-elnusa-tbk",
    "https://www.kompasiana.com/rikarosilawati1341/650657186e14f15604652852/pentingnya-belajar-sastra",
    "https://www.kompasiana.com/suprihadi48660/65064ae76e14f150a1547a32/sosialisasi-eyd-bagi-pengawas-dan-penilik-sekolah",
    "https://www.kompasiana.com/sema-fpsikologi3509/6505d7fc6e14f15e4f18e342/segerakan-infomatis-pembangun-sinergitas",
    "https://www.kompasiana.com/anwarngabdullah/6505ced908a8b56b023b7f23/sholat-dhuha-berjamaah-sebagai-upaya-penguatan-religiusitas-siswa-smk-penerbangan-aag-adisutjipto-yogyakarta",
    "https://www.kompasiana.com/simatupang65928/650522826e14f17b397ee6b2/dampak-perjanjian-camp-david-terhadap-negara-palestina",
    "https://www.kompasiana.com/yunusilhamiputra/6505f90d6e14f16c1c02d9b2/rebutan-basis-nu",
    "https://www.kompasiana.com/herguna86/6505ea6908a8b525ca0b00a2/bhinneka-tunggal-ika-berdampingan-dalam-keberagaman",
    "https://www.kompasiana.com/almubarak/65059c6c6e14f110cd6990d2/prabowo-subianto-pernah-dikebiri-saat-bertugas-di-timor-timur-benarkah",
    "https://www.kompasiana.com/owenjuve/65058ac608a8b510f2157b82/membaca-arah-politik-ridwan-kamil",
    "https://www.kompasiana.com/tomisatryatomo7169/650472664addee61d5084dc2/genap-22-tahun-bagaimana-partai-demokrat-di-mata-media-dan-netizen",
    "https://www.kompasiana.com/saodahnurhabibah3693/65062bb24addee0b807ceb12/memahami-bullying-di-lingkungan-keluarga",
    "https://www.kompasiana.com/nitakristantinoer/650551916e14f1776f266c83/catatan-reflektif-perundungan-dalam-dua-sisi",
    "https://www.kompasiana.com/henisilalahi9126/6505060a6e14f1516e0be1c4/waspada-family-scapegoat-abuse-fsa-di-sekitar-kita-bagian-1",
    "https://www.kompasiana.com/naniherawati5566/64f96ae7e2c0f912b344a242/sosialisasi-pengaruh-bullying-terhadap-kemampuan-sosial-di-smk-sasmita-jaya-2",
    "https://www.kompasiana.com/ndrewp2335/64e04adc18333e713c70cdf2/rekayasa-perangkat-lunak-pengertian-dan-penjelasannya",
    "https://www.kompasiana.com/rahmadwahyujadmiko2707/64b5782b08a8b57b1c04a7a2/system-defelpoment-life-cycle-dengan-metode-waterfall",
    "https://www.kompasiana.com/sitiaisyah110385/65014d2b1769831300646e42/pentingnya-peningkatan-kompetensi-operator-sekolah",
    "https://www.kompasiana.com/monikatulenan5760/6502acd508a8b51f5a0a6724/teknologi-perkembangan-zaman-dan-model-pembelajaran",
    "https://www.kompasiana.com/desytawangkai9434/65027ce408a8b57be878bf12/perkembangan-teknologi-media-dan-metode-pembelajaran",
    "https://www.kompasiana.com/zaimzam/64ed61f94addee11cb52ea72/membangun-generasi-digital-melalui-kursus-komputer-di-ifan-jaya-school",
    "https://www.kompasiana.com/henisilalahi9126/6505060a6e14f1516e0be1c4/waspada-family-scapegoat-abuse-fsa-di-sekitar-kita-bagian-1",
    "https://www.kompasiana.com/naniherawati5566/64f96ae7e2c0f912b344a242/sosialisasi-pengaruh-bullying-terhadap-kemampuan-sosial-di-smk-sasmita-jaya-2",
    "https://www.kompasiana.com/ndrewp2335/64e04adc18333e713c70cdf2/rekayasa-perangkat-lunak-pengertian-dan-penjelasannya",
    "https://www.kompasiana.com/rahmadwahyujadmiko2707/64b5782b08a8b57b1c04a7a2/system-defelpoment-life-cycle-dengan-metode-waterfall",
    "https://www.kompasiana.com/sitiaisyah110385/65014d2b1769831300646e42/pentingnya-peningkatan-kompetensi-operator-sekolah",
    "https://www.kompasiana.com/monikatulenan5760/6502acd508a8b51f5a0a6724/teknologi-perkembangan-zaman-dan-model-pembelajaran",
    "https://www.kompasiana.com/desytawangkai9434/65027ce408a8b57be878bf12/perkembangan-teknologi-media-dan-metode-pembelajaran",
    "https://www.kompasiana.com/zaimzam/64ed61f94addee11cb52ea72/membangun-generasi-digital-melalui-kursus-komputer-di-ifan-jaya-school",
    "https://www.kompasiana.com/shella56908/63940a9e4addee7f9519d122/bisnis-online-di-era-digital-dan-pandemi-covid-19",
    "https://www.kompasiana.com/elenaberliana1317/650198cc4addee642622c2f2/perkembangan-teknologi-dan-ekonomi-ai-dibidang-farmasi",
    "https://www.kompasiana.com/31_sutanrajwanayyara8076/650192dc1769837c8160e092/pentingnya-teknologi-informasi-pada-bidang-farmasi",
    "https://www.kompasiana.com/auliasafiraputri9057/64fe5ba508a8b5569a33fc42/teknologi-informasi-dalam-bidang-farmasi",
    "https://www.kompasiana.com/nadiafaradila8555/64fe592508a8b5780b2ec3a2/pengembangan-perangkat-lunak-pengolahan-data-farmasi-pada-klinik-kesehatan",
    "https://www.kompasiana.com/auliasafiraputri9057/64fd6bc64addee194502a602/teknologi-informasi-dalam-bidang-farmasi",
    "https://www.kompasiana.com/hafizhahnaf4857/64ea05944addee44095428c2/dampak-pengembangan-teknologi-kedokteran-dalam-transformasi-dan-layanan",
    "https://www.kompasiana.com/didikfatmoko3878/64e5d04408a8b52e872f7052/mengenal-e3d-teknologi-cetak-3d-organ-manusia-untuk-bantu-edukasi-di-bidang-kedokteran",
    "https://www.kompasiana.com/e1rays/64e4f5d908a8b535a17f7fe2/pengembangan-teknologi-kedokteran-untuk-teknologi-kesehatan-yang-presisi",
    "https://www.kompasiana.com/intansaputri8311/64e4da8618333e765c2a0fb2/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/aulexandern/64e4c1b418333e7683198a63/transformasi-teknologi-kesehatan-yang-tidak-merata",
    "https://www.kompasiana.com/gabriella7326/64e4b57708a8b51787260502/pengembangan-teknologi-kedokteran-di-era-industri-4-0",
    "https://www.kompasiana.com/annisatartilla2485/64e38a6808a8b523c437ea32/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/muhammadkemal2038/64e388bb18333e1a7b11f4a4/pengembangan-teknologi-kedokteran-untuk-tranformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/nvttdy/64e388204addee240e5b2254/pengembangan-teknologi-kedokteran-guna-menciptakan-pelayanan-medis-minim-invasif",
    "https://www.kompasiana.com/nancynaila/64e238244addee02f56a1a22/transformasi-teknologi-kesehatan-dari-pengembangan-teknologi-kedokteran",
    "https://www.kompasiana.com/nauraajn/64e20c9e18333e5f4e21b194/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan-dari-pandangan-kontra",
    "https://www.kompasiana.com/sylviasfm/64d8e7564addee46d80ca8c2/mahasiswa-kkn-dan-masyarakat-lumeneng-perang-melawan-lingkaran-setan-stunting",
    "https://www.kompasiana.com/monikatulenan5760/6502acd508a8b51f5a0a6724/teknologi-perkembangan-zaman-dan-model-pembelajaran",
    "https://www.kompasiana.com/desytawangkai9434/65027ce408a8b57be878bf12/perkembangan-teknologi-media-dan-metode-pembelajaran",
    "https://www.kompasiana.com/zaimzam/64ed61f94addee11cb52ea72/membangun-generasi-digital-melalui-kursus-komputer-di-ifan-jaya-school",
    "https://www.kompasiana.com/shella56908/63940a9e4addee7f9519d122/bisnis-online-di-era-digital-dan-pandemi-covid-19",
    "https://www.kompasiana.com/elenaberliana1317/650198cc4addee642622c2f2/perkembangan-teknologi-dan-ekonomi-ai-dibidang-farmasi",
    "https://www.kompasiana.com/31_sutanrajwanayyara8076/650192dc1769837c8160e092/pentingnya-teknologi-informasi-pada-bidang-farmasi",
    "https://www.kompasiana.com/auliasafiraputri9057/64fe5ba508a8b5569a33fc42/teknologi-informasi-dalam-bidang-farmasi",
    "https://www.kompasiana.com/nadiafaradila8555/64fe592508a8b5780b2ec3a2/pengembangan-perangkat-lunak-pengolahan-data-farmasi-pada-klinik-kesehatan",
    "https://www.kompasiana.com/auliasafiraputri9057/64fd6bc64addee194502a602/teknologi-informasi-dalam-bidang-farmasi",
    "https://www.kompasiana.com/hafizhahnaf4857/64ea05944addee44095428c2/dampak-pengembangan-teknologi-kedokteran-dalam-transformasi-dan-layanan",
    "https://www.kompasiana.com/didikfatmoko3878/64e5d04408a8b52e872f7052/mengenal-e3d-teknologi-cetak-3d-organ-manusia-untuk-bantu-edukasi-di-bidang-kedokteran",
    "https://www.kompasiana.com/e1rays/64e4f5d908a8b535a17f7fe2/pengembangan-teknologi-kedokteran-untuk-teknologi-kesehatan-yang-presisi",
    "https://www.kompasiana.com/intansaputri8311/64e4da8618333e765c2a0fb2/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/aulexandern/64e4c1b418333e7683198a63/transformasi-teknologi-kesehatan-yang-tidak-merata",
    "https://www.kompasiana.com/gabriella7326/64e4b57708a8b51787260502/pengembangan-teknologi-kedokteran-di-era-industri-4-0",
    "https://www.kompasiana.com/annisatartilla2485/64e38a6808a8b523c437ea32/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/muhammadkemal2038/64e388bb18333e1a7b11f4a4/pengembangan-teknologi-kedokteran-untuk-tranformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/nvttdy/64e388204addee240e5b2254/pengembangan-teknologi-kedokteran-guna-menciptakan-pelayanan-medis-minim-invasif",
    "https://www.kompasiana.com/nancynaila/64e238244addee02f56a1a22/transformasi-teknologi-kesehatan-dari-pengembangan-teknologi-kedokteran",
    "https://www.kompasiana.com/nauraajn/64e20c9e18333e5f4e21b194/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan-dari-pandangan-kontra",
    "https://www.kompasiana.com/sylviasfm/64d8e7564addee46d80ca8c2/mahasiswa-kkn-dan-masyarakat-lumeneng-perang-melawan-lingkaran-setan-stunting",
    "https://www.kompasiana.com/almubarak/65059c6c6e14f110cd6990d2/prabowo-subianto-pernah-dikebiri-saat-bertugas-di-timor-timur-benarkah",
    "https://www.kompasiana.com/owenjuve/65058ac608a8b510f2157b82/membaca-arah-politik-ridwan-kamil",
    "https://www.kompasiana.com/tomisatryatomo7169/650472664addee61d5084dc2/genap-22-tahun-bagaimana-partai-demokrat-di-mata-media-dan-netizen",
    "https://www.kompasiana.com/saodahnurhabibah3693/65062bb24addee0b807ceb12/memahami-bullying-di-lingkungan-keluarga",
    "https://www.kompasiana.com/nitakristantinoer/650551916e14f1776f266c83/catatan-reflektif-perundungan-dalam-dua-sisi",
    "https://www.kompasiana.com/shella56908/63940a9e4addee7f9519d122/bisnis-online-di-era-digital-dan-pandemi-covid-19",
    "https://www.kompasiana.com/elenaberliana1317/650198cc4addee642622c2f2/perkembangan-teknologi-dan-ekonomi-ai-dibidang-farmasi",
    "https://www.kompasiana.com/31_sutanrajwanayyara8076/650192dc1769837c8160e092/pentingnya-teknologi-informasi-pada-bidang-farmasi",
    "https://www.kompasiana.com/auliasafiraputri9057/64fe5ba508a8b5569a33fc42/teknologi-informasi-dalam-bidang-farmasi",
    "https://www.kompasiana.com/nadiafaradila8555/64fe592508a8b5780b2ec3a2/pengembangan-perangkat-lunak-pengolahan-data-farmasi-pada-klinik-kesehatan",
    "https://www.kompasiana.com/auliasafiraputri9057/64fd6bc64addee194502a602/teknologi-informasi-dalam-bidang-farmasi",
    "https://www.kompasiana.com/hafizhahnaf4857/64ea05944addee44095428c2/dampak-pengembangan-teknologi-kedokteran-dalam-transformasi-dan-layanan",
    "https://www.kompasiana.com/didikfatmoko3878/64e5d04408a8b52e872f7052/mengenal-e3d-teknologi-cetak-3d-organ-manusia-untuk-bantu-edukasi-di-bidang-kedokteran",
    "https://www.kompasiana.com/e1rays/64e4f5d908a8b535a17f7fe2/pengembangan-teknologi-kedokteran-untuk-teknologi-kesehatan-yang-presisi",
    "https://www.kompasiana.com/intansaputri8311/64e4da8618333e765c2a0fb2/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/aulexandern/64e4c1b418333e7683198a63/transformasi-teknologi-kesehatan-yang-tidak-merata",
    "https://www.kompasiana.com/gabriella7326/64e4b57708a8b51787260502/pengembangan-teknologi-kedokteran-di-era-industri-4-0",
    "https://www.kompasiana.com/annisatartilla2485/64e38a6808a8b523c437ea32/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/muhammadkemal2038/64e388bb18333e1a7b11f4a4/pengembangan-teknologi-kedokteran-untuk-tranformasi-teknologi-kesehatan",
    "https://www.kompasiana.com/nvttdy/64e388204addee240e5b2254/pengembangan-teknologi-kedokteran-guna-menciptakan-pelayanan-medis-minim-invasif",
    "https://www.kompasiana.com/nancynaila/64e238244addee02f56a1a22/transformasi-teknologi-kesehatan-dari-pengembangan-teknologi-kedokteran",
    "https://www.kompasiana.com/nauraajn/64e20c9e18333e5f4e21b194/pengembangan-teknologi-kedokteran-untuk-transformasi-teknologi-kesehatan-dari-pandangan-kontra",
    "https://www.kompasiana.com/sylviasfm/64d8e7564addee46d80ca8c2/mahasiswa-kkn-dan-masyarakat-lumeneng-perang-melawan-lingkaran-setan-stunting",
    "https://www.kompasiana.com/almubarak/65059c6c6e14f110cd6990d2/prabowo-subianto-pernah-dikebiri-saat-bertugas-di-timor-timur-benarkah",
    "https://www.kompasiana.com/owenjuve/65058ac608a8b510f2157b82/membaca-arah-politik-ridwan-kamil",
    "https://www.kompasiana.com/tomisatryatomo7169/650472664addee61d5084dc2/genap-22-tahun-bagaimana-partai-demokrat-di-mata-media-dan-netizen",
    "https://www.kompasiana.com/saodahnurhabibah3693/65062bb24addee0b807ceb12/memahami-bullying-di-lingkungan-keluarga",
    "https://www.kompasiana.com/nitakristantinoer/650551916e14f1776f266c83/catatan-reflektif-perundungan-dalam-dua-sisi",
    "https://www.kompasiana.com/henisilalahi9126/6505060a6e14f1516e0be1c4/waspada-family-scapegoat-abuse-fsa-di-sekitar-kita-bagian-1",
    "https://www.kompasiana.com/naniherawati5566/64f96ae7e2c0f912b344a242/sosialisasi-pengaruh-bullying-terhadap-kemampuan-sosial-di-smk-sasmita-jaya-2",
    "https://www.kompasiana.com/ndrewp2335/64e04adc18333e713c70cdf2/rekayasa-perangkat-lunak-pengertian-dan-penjelasannya",
    "https://www.kompasiana.com/rahmadwahyujadmiko2707/64b5782b08a8b57b1c04a7a2/system-defelpoment-life-cycle-dengan-metode-waterfall",
    "https://www.kompasiana.com/sitiaisyah110385/65014d2b1769831300646e42/pentingnya-peningkatan-kompetensi-operator-sekolah",
    "https://www.kompasiana.com/ariffadhil/65095cda4addee19d630a152/polusi-udara-di-jakarta-semakin-tebal-masyarakat-harus-meningkatkan-kesadaran-kualitas-udara-demi-kesehatan",
    "https://www.kompasiana.com/auliafadiah1564/6506f6b508a8b522d31894c2/polusi-makin-tebal-ini-persiapan-yang-bijak-menghadapi-polusi-dan-cuaca-panas",
    "https://www.kompasiana.com/dwitarupansa/6505d4de4addee1eba00aaf2/udara-jakarta-makin-parah-kesehatan-kita-terancam",
    "https://www.kompasiana.com/nathanaelrestianto/65040e0f08a8b515473abd52/polusi-kian-memburuk-kian-membaik",
    "https://www.kompasiana.com/imamkurniawan9563/6503e61408a8b51b0d0fc8e2/polusi-udara-jakarta-ancaman-nyata-yang-mengintai-warga-jakarta",
    "https://www.kompasiana.com/paltihutabarat/6503dae008a8b5224b5b9b22/polusi-berkurang-langit-biru-sudah-mulai-sering-kelihatan-apa-kebijakan-pemerintah-selanjutnya",
    "https://www.kompasiana.com/risya1234/6501857108a8b56c607b3bc2/lampu-lampu-yang-masih-lama-menyalanya",
    "https://www.kompasiana.com/adibnrtwn4376/65010bc14addee03f161db63/mengatasi-polusi-udara-yang-sedang-melanda-indonesia-khususnya-di-jakarta-yang-semakin-tinggi",
    "https://www.kompasiana.com/muhammadrezanuramin4429/6500f416e1a16766fa2a5c62/polusi-udara-jakarta",
    "https://www.kompasiana.com/dannyfebrian2170/64fe763708a8b50b013b6fb2/polusi-jakarta-bikin-rentan-flu-benarkah",
    "https://www.kompasiana.com/riza39427/6425242f4943472e16710ac2/fakta-tentang-olahraga-yang-dapat-mengurangi-tingkat-stress",
    "https://www.kompasiana.com/laily110901/61d1e28016671775bb738b42/tentang-olahraga-dayung-di-indonesia-canoeing",
    "https://www.kompasiana.com/gordi/57f818ae127f61cf230d0fcb/tiga-pemimpin-dunia-bicara-tentang-olahraga-dan-kemanusiaan",
    "https://www.kompasiana.com/wil/5742caae9497734c08e0b767/tentang-olahraga-yang-menyehatkan-fisik-dan-jiwa",
    "https://www.kompasiana.com/ludfinamailaniputri5484/64e4aa4608a8b532c27c3c92/mengenal-lebih-dekat-tentang-kesehatan-mental"
    "https://www.kompasiana.com/aninditazeriana1505/649082f74addee78ce1dd662/menyuarakan-kesadaran-meningkatkan-kesadaran-tentang-kesehatan-mental-di-lingkungan-sekitar",
    "https://www.kompasiana.com/rachmadany7/641f1500d3aa0f0ce54a1a22/membangun-kesadaran-tentang-kesehatan-mental-sebagai-salah-satu-bentuk-pemberdayaan-remaja",
    "https://www.kompasiana.com/khusnul29237/63d36ab004dff02a6b703342/pandangan-psikologi-agama-tentang-kesehatan-mental",
    "https://www.kompasiana.com/riapwindhu/64d9f5e508a8b53e787a0462/lebih-hijau-dn-makin-cantik-nggak-cukup-sehari-bercerita-tentang-indonesia-di-taman-mini",
    "https://www.kompasiana.com/gaganawati/649ffe55e1a1671d4f3f8202/mengajak-orang-asing-untuk-talkshow-cerita-tentang-indonesia",
    "https://www.kompasiana.com/andri18517/6310c05bc3bdbf1c5f038582/pro-kontra-kebijakan-kampanye-di-kampus-melihat-seberapa-jauh-pemikiran-mahasiswa-tentang-politik-indonesia",
    "https://www.kompasiana.com/mundursatuaje7380/6497cb574addee6aa410c3a2/generasi-muda-menjadi-peran-penting-tentang-politik-yang-ada-di-media-sosial",
    "https://www.kompasiana.com/antoniusgumuk3053/648955fb4addee34b8511d72/membangun-pemikiran-masyarakat-tentang-politik",
    "https://www.kompasiana.com/mohamadmiftachmegantoro8687/615954fe2881757bc6196574/dibalik-fakta-cerita-dan-stigma-tentang-kuliah-di-pkn-stan",
    "https://www.kompasiana.com/yolenta070998/60d9772406310e63b466f7b2/opini-mahasiswi-tentang-kuliah-online-yang-masih-berlanjut",
    "https://www.kompasiana.com/noto69047/5facbc38d541df1f5c481232/mau-tau-ada-hal-positif-tentang-corona",
    "https://www.kompasiana.com/ypiwiratatabuana/5e9ad46a097f3647de14fc54/apa-yang-kita-ketahui-tentang-coronavirus-2",
    "https://www.kompasiana.com/negarabaru/5e970049097f3662204893f3/hoax-kominfo-tentang-corona",
    "https://www.kompasiana.com/anunganindita6820/5e5b416b097f36625f093424/lindungi-diri-dan-orang-orang-tersayang-yang-harus-kita-ketahui-tentang-corona-virus",
    "https://www.kompasiana.com/karlogeofandy4894/5e480ac0097f36600c60d8b2/14-mitos-tentang-coronavirus-hoax-atau-fakta",
    "https://www.kompasiana.com/rahmatridho/650cf53e856e0a2a12484f02/menggagas-kehutanan-yang-cerdas-iklim-peran-kunci-dalam-mengatasi-perubahan-iklim-global",
    "https://www.kompasiana.com/josex1/650c4dfe856e0a245b645312/tipe-ekosistem",
    "https://www.kompasiana.com/rahmatridho/6507bc336e14f166250b99d2/mengubah-sungai-menjadi-sumber-keberlanjutan-pangan-rahasia-restorasi-hutan-tepi-sungai-di-indonesia",
    "https://www.kompasiana.com/umsidamenyapa5821/6507f8d66e14f16a7a76ca92/suhu-bumi-tahun-2023-terpanas-sepanjang-sejarah-ini-kata-pakar-umsida",
    "https://www.kompasiana.com/c22eindahazilah2796/6503ce0c08a8b54efd5a6332/kebakaran-sampah-mengakibatkan-polusi-udara",
    "https://www.kompasiana.com/kurniaalbarkati9396/65030b9ee3b78f52040ed2e2/optimalisasi-potensi-hutan-produksi-dengan-konsep-pelestarian-secara-berkelanjutan",
    "https://www.kompasiana.com/ameliaroby0610/64ffcf5208a8b53d3541c0e2/peran-polisi-hutan-dalam-melindungi-dan-pencegahan-tindakan-kejahatan-di-dalam-hutan",
    "https://www.kompasiana.com/wiwin10587/64efd18f18333e59313e8782/kemarau-dan-polusi",
    "https://www.kompasiana.com/dianfbrynti6271/650dacf608a8b5508e019c02/apakah-peran-sistem-ekonomi-islam-dalam-masyarakat-itu-penting",
    "https://www.kompasiana.com/nia23815/650d8340856e0a10ab68acc2/perspektif-dan-peran-sosiologi-ekonomi-dalam-pembangunan-ekonomi-masyarakat",
    "https://www.kompasiana.com/fitriani51727/650d5ad24addee038022fca2/peran-sosiologi-ekonomi-dalam-pembangunan-ekonomi-masyarakat",
    "https://www.kompasiana.com/mardasasmita6162/650d353a4addee282512ef02/system-ekonomi-dan-ilmu-pengetahuan-berdasar-agama-islam-di-indonesia",
    "https://www.kompasiana.com/nurfitriani6207/650d094808a8b53c0d483e42/agama-sebagai-faktor-penggerak-atau-dasar-dalam-ekonomi-dan-ilmu-pengetahuan",
    "https://www.kompasiana.com/rovinsrn8626/650d0011856e0a58f54b6a92/transformasi-ekonomi-global-keterkaitan-antara-gdp-dan-gnp-dalam-era-globalisasi",
    "https://www.kompasiana.com/santriumphs/650cfb29856e0a547b7753d2/liquidity-hunt",
    "https://www.kompasiana.com/kucingpasar4540/650cdba34addee29f84248d4/cara-meredam-persaingan-bisnis",
    "https://www.kompasiana.com/diniapriliawardani4321/650cccd94addee4228268e52/dampak-krisis-ekonomi-terhadap-gdp-dan-dnp-di-wilayah-negara-berkembang",
    "https://www.kompasiana.com/hikmaeksyfour/650c6beb4addee6e1938ef42/sosiologi-kepentingan-interest-dalam-tindakan-ekonomi",
    "https://www.kompasiana.com/santichris2212/650c5ea2856e0a7bc907aec2/gentrifikasi-dapat-meningkatkan-pertumbuhan-ekonomi-bersama",
    "https://www.kompasiana.com/hasiah246/650c4e264addee2de33de312/perspektif-dan-peran-sosiologi-ekonomi-dalam-pembangunan-ekonomi-masyarakat"
]

In [5]:
skipped_urls = []
clean_documents = []
for url in urls:
    # Kirim permintaan GET
    response = requests.get(url)

    # Parse konten dengan BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Temukan elemen dengan atribut itemprop="articleBody"
    article_body = soup.find(itemprop='articleBody')
    
    # Jika elemen tidak ditemukan, lewati url dan tambahkan ke list
    if article_body is None:
        skipped_urls.append(url)
        continue

    # Dapatkan teks dari setiap elemen <p> dan simpan ke dalam list
    paragraphs = [p.get_text() for p in article_body.find_all('p')]
    
    # Gabungkan list menjadi satu string, dengan setiap paragraf dipisahkan oleh baris baru
    text = '\n'.join(paragraphs) 
    
     # Hapus tanda baca dan karakter non-alfabet
    text = re.sub(r'[^\w\s]', ' ', text)

    # Case folding
    text = text.lower()
    
    # Ekstrak judul halaman sebagai nama file (Anda mungkin perlu menyesuaikan ini tergantung pada struktur halaman web)
    title = soup.title.string

    # Buat versi singkat dari judul untuk digunakan sebagai nama file
    short_title = title[:50]  # Ambil 50 karakter pertama dari judul

    # Ganti karakter yang tidak valid dalam nama file
    filename = "".join(c for c in short_title if c.isalnum() or c.isspace()).rstrip() + ".txt"

    # Tentukan direktori tempat Anda ingin menyimpan file
    directory = "C:/Users/ahini/Downloads/projek_pi_4/tes/"

    # Gabungkan direktori dan nama file untuk mendapatkan path lengkap ke file
    file_path = os.path.join(directory, filename)

    # Simpan teks ke file .txt
    with open(file_path, 'w') as f:
        f.write(text)